# Initial ideas

<b>Language model fine-tuned on statements and values to provide chatbot conversations with humans with different values</b>


1. Train a BERT (BART?) model to classify values based on prompt sentences
2. Apply reinforcement learning using the learned value model to fine-tune language model to predict next sentence in sequence.
3. Can we use this to create a chat bot or do we need to create our own language model and architecture?

In [6]:
import pandas as pd
import os
import numpy as np

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [14]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 10)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Try to do a simple forward pass

In [4]:
# Load dataset
folder_name = 'v0.3_original'
df_ACHIEVEMENT = pd.read_csv(os.path.join(folder_name, 'ACHIEVEMENT.csv'))
s = df_ACHIEVEMENT['scenario'][0]

# Forward pass
with torch.no_grad():
    inp = tokenizer(s, return_tensors='pt')
    out = model(**inp)
    print(out)


SequenceClassifierOutput(loss=None, logits=tensor([[0.0537]]), hidden_states=None, attentions=None)


## Pre-process dataset
1. Load into dataframe
2. Tokenize the data to fit input format for bert model

<b>Problems</b>
How to handle zero values? Do we need to use the value in the predictions?

In [5]:
# Load the data
VALUES = ['ACHIEVEMENT', 'BENEVOLENCE', 'CONFORMITY', 'HEDONISM', 'POWER', 'SECURITY', 'SELF-DIRECTION', 'STIMULATION', 'TRADITION', 'UNIVERSALISM']

In [8]:
class ValueDataset(Dataset):
    
    def __init__(self, tokenizer): 
        df = self._load_data(VALUES)
        self.scenarios = df['scenario'].values.tolist()
        self.values = df['label'].values.tolist()
        self.N = df.shape[0]

        inp = tokenizer(self.scenarios, return_tensors='pt', padding=True, truncation=True)
        self.input_ids = inp.get('input_ids')
        self.attention_mask = inp.get('attention_mask')
        self.token_type_ids = inp.get('token_type_ids')

        self.y = torch.from_numpy(df[VALUES].values) # Values
    
    def __getitem__(self, index):
        return self.input_ids[index], self.attention_mask[index], self.token_type_ids[index], self.y[index]

    def __len__(self):
        return self.N

    def _load_data(self, values):

        df = pd.DataFrame()

        for value in values:
            df_value = pd.read_csv(os.path.join(folder_name, value + '.csv'))
            df_value = df_value.rename({'label' : value}, axis = 'columns')
            df_value['label'] = value
            df = pd.concat([df, df_value])

        df.fillna(0, inplace=True)
        df.reset_index(inplace=True)
        return df

dataset = ValueDataset(AutoTokenizer.from_pretrained('bert-base-uncased'))
dataloader = DataLoader(dataset=dataset, batch_size=10, shuffle=True)
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 10)

# Try to conduct a batched forward pass
with torch.no_grad():
    input_ids, attention_mask, token_types_ids, targets = next(iter(dataloader)) 
    inp = {'input_ids' : input_ids, 'attention_mask' : attention_mask, 'token_type_ids' : token_types_ids}
    output = model(**inp)
    assert output.logits.size() == targets.size()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def load_data(values):

        df = pd.DataFrame()

        for value in values:
            df_value = pd.read_csv(os.path.join(folder_name, value + '.csv'))
            df_value = df_value.rename({'label' : value}, axis = 'columns')
            df_value['label'] = value
            df = pd.concat([df, df_value])

        df.fillna(0, inplace=True)
        df.reset_index(inplace=True)
        return df


df = load_data(VALUES)


In [8]:
train_split = .8
N = df.shape[0]

train_dataset = Dataset.from_dict(tokenizer(df['scenario'][0:int(N * train_split)].values.tolist(), padding = True, truncation = True, return_tensors = 'pt'))
validation_dataset = Dataset.from_dict(tokenizer(df['scenario'][int(N * train_split):].values.tolist(), padding = True, truncation = True, return_tensors = 'pt'))

# Train model
1. Train model to score values based on different values

In [21]:
model()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [20]:
df.iloc[0]

index                                                             0
Unnamed: 0                                                        0
uid                                                            2635
scenario          Tomorrow I will audition my singing in a talen...
ACHIEVEMENT                                                     1.0
label                                                   ACHIEVEMENT
BENEVOLENCE                                                     0.0
CONFORMITY                                                      0.0
HEDONISM                                                        0.0
POWER                                                           0.0
SECURITY                                                        0.0
SELF-DIRECTION                                                  0.0
STIMULATION                                                     0.0
TRADITION                                                       0.0
UNIVERSALISM                                    

In [15]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset, load_dataset

trainer = Trainer(
    model = AutoModelForSequenceClassification('bert-base-uncased', num_labels = 10), 
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,
    tokenizer = tokenizer
)

trainer.train()

OSError: AutoModelForSequenceClassification is designed to be instantiated using the `AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path)` or `AutoModelForSequenceClassification.from_config(config)` methods.

In [289]:
out.logits

tensor([[ 0.5816, -0.1002, -0.8615, -0.3651,  0.3426,  0.1915,  0.4009,  0.2929,
          0.1102, -0.0562],
        [ 0.6290, -0.2749, -0.5745, -0.1988,  0.1777,  0.0832,  0.2657,  0.2745,
          0.0169,  0.0462],
        [ 0.5651,  0.0624, -0.7030, -0.1752,  0.1315, -0.0870, -0.0310,  0.2596,
         -0.0729,  0.1607],
        [ 0.6077, -0.0241, -0.7932, -0.3335,  0.3101,  0.1662,  0.3302,  0.3789,
          0.0865,  0.0444],
        [ 0.6952, -0.0650, -0.9161, -0.2959,  0.3510,  0.1231,  0.2523,  0.3453,
          0.0801,  0.1204],
        [ 0.6582, -0.2365, -0.7176, -0.3096,  0.2731,  0.0280,  0.1341,  0.2272,
         -0.0747,  0.1925],
        [ 0.6931, -0.1226, -0.7247, -0.2466,  0.2624, -0.0261,  0.1203,  0.2805,
         -0.1136,  0.1546],
        [ 0.6367, -0.1960, -0.5345, -0.1218,  0.2135, -0.0971,  0.2804,  0.2762,
         -0.0388, -0.0614],
        [ 0.6698, -0.0232, -0.8080, -0.2781,  0.3113,  0.0392,  0.2370,  0.2740,
          0.0713,  0.0606],
        [ 0.6423, -

In [287]:
loss_function = F.mse_loss
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)
epochs = 10

for epoch in range(epochs):
    input_ids, attention_mask, token_types_ids, targets = next(iter(dataloader)) 
    inp = {'input_ids' : input_ids, 'attention_mask' : attention_mask, 'token_type_ids' : token_types_ids}
    out = model(**inp)
    
    optimizer.zero_grad()
    loss = loss_function(out.logits, targets)
    loss.backward()
    optimizer.step()
    



RuntimeError: Found dtype Double but expected Float